# Installs, Imports

In [1]:
'''NOT SURE WE ARE USING THIS. MAY DELETE AFTER DEBUGGING KAH GEO-ENCODING SECTION'''

# import geoencoding libraries
# !pip install geopandas
# !pip install geopy
# !pip install folium

'NOT SURE WE ARE USING THIS. MAY DELETE AFTER DEBUGGING KAH GEO-ENCODING SECTION'

In [2]:
'''
This is based on an NYU presentation from Nicholas Wolf and Vicky Steeves-
Python for Harvesting Data on the Web - https://nyu-dataservices.github.io/DataHarvesting-Python/
'''

import pandas as pd
import numpy as np
from numpy import array 
from numpy import argmax 
from sklearn.preprocessing import LabelEncoder 
import datetime
from sklearn.model_selection import train_test_split
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

# Reading and Filtering Data

In [3]:
'''
Read in datasets from .csv's saved in same directory
Change all column names to lowercase

Initiation.csv downloaded Jan 5, 2020
from https://datacatalog.cookcountyil.gov/Courts/Initiation/7mck-ehwz

Dispositions.csv downloaded Jan 5, 2020
from https://datacatalog.cookcountyil.gov/Courts/Dispositions/apwk-dzx8

MHI_true.csv created via MHI_building.ipynb notebook
'''

initiation = pd.read_csv('Initiation.csv')
initiation.columns = [x.lower() for x in initiation.columns]

dispositions = pd.read_csv('Dispositions.csv')
dispositions.columns = [x.lower() for x in dispositions.columns]

sentencing = pd.read_csv('Dispositions.csv')
sentencing.columns = [x.lower() for x in sentencing.columns]

MHI_true = pd.read_csv('MHI_true.csv')
MHI_true.columns = [x.lower() for x in MHI_true.columns]

/Users/AKR/HackLife/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
'''
Filter initiation dataset 
Keep only the initiation records with
case_participant_id's that exist in either sentencing or dispositions
'''

#save cp id values as lists
init_ids = list(initiation.case_participant_id.values)
dispo_ids = list(dispositions.case_participant_id.values)
sent_ids = list(sentencing.case_participant_id.values)

#create list of unique cp ids that are in both init & dispo
ids_to_keep = list(set(init_ids).intersection(set(dispo_ids).union(set(sent_ids))))

#create filtered df with only records from the intersection of cp ids
init_filtered = initiation[initiation['case_participant_id'].isin(ids_to_keep)]

del dispositions
del sentencing

In [5]:
#CHECKPOINT
# need to run the prior if we don't have 'init_filtered.csv' - amber notes
#start from here if don't want to rerun everything before?
#move to new notebook?
#init_filtered = pd.read_csv('init_filtered.csv', index_col=0)

# Data Cleaning

In [6]:
#create new df to work on for processing/cleaning
init_clean = init_filtered.copy()

del init_filtered

In [7]:
'''
Cleaning age_at_incident column
'''

#print unique values of age
print('Unique age values: ', init_clean['age_at_incident'].unique())

#turn nan to '0' in order to process into ints
init_clean.loc[init_clean['age_at_incident'].isnull(), 'age_at_incident'] = '0'

#change type to int
init_clean.age_at_incident = init_clean.age_at_incident.astype(int)

#look at unique values again
print('Unique age values after cleaning: ', init_clean['age_at_incident'].unique())

#how many values do we think are messed up?
print('Number of records with null age: ', len(init_clean[init_clean['age_at_incident']==0]))
print('Number of records with age > 100: ', len(init_clean[init_clean['age_at_incident']>100]))


Unique age values:  [ 22.  nan  29.  34.  27.  41.  17.  20.  25.  59.  19.  43.  31.  23.
  32.  18.  30.  21.  28.  39.  26.  46.  37.  44.  35.  49.  42.  24.
  52.  51.  38.  33.  57.  58.  45.  48.  50.  65.  47.  53.  40.  60.
  36.  56.  55.  67.  64.  54.  63.  70.  62.  61.  71.  66.  74.  75.
  69.  68.  73.  72. 111.  85.  84.  86.  78.  77.  76.  79. 156.  81.
  80.  82.  83. 112. 113. 114. 120. 127. 115. 130. 116.  96.  87. 117.
 125. 118. 119.]
Unique age values after cleaning:  [ 22   0  29  34  27  41  17  20  25  59  19  43  31  23  32  18  30  21
  28  39  26  46  37  44  35  49  42  24  52  51  38  33  57  58  45  48
  50  65  47  53  40  60  36  56  55  67  64  54  63  70  62  61  71  66
  74  75  69  68  73  72 111  85  84  86  78  77  76  79 156  81  80  82
  83 112 113 114 120 127 115 130 116  96  87 117 125 118 119]
Number of records with null age:  10954
Number of records with age > 100:  37


In [8]:
'''
Create 2 new age-related binary features: age > 100, age_ unknown
Use median to impute missing or presumed-incorrect age values
'''

#creating new binary colmns for age over 100 and age unknown (1 = true)
init_clean['age_over_100'] = (init_clean.age_at_incident > 100).astype(int) 
init_clean['age_unknown'] = (init_clean.age_at_incident == 0).astype(int)


#replacing > 100 and 0 values with nan, calculating median, then changing nan to mean
init_clean.age_at_incident.replace(0, np.NaN, inplace=True)
a = np.array(init_clean['age_at_incident'].values.tolist())
init_clean['age_at_incident'] = np.where(a > 100, np.nan, a).tolist()
print('Number of records with missing/incorrect age: ', init_clean.age_at_incident.isna().sum())
median = init_clean['age_at_incident'].median()
print('Original median age: ', median)
print('Original mean age: ', init_clean['age_at_incident'].mean())
init_clean.age_at_incident.replace(np.NaN, median, inplace=True) 
print('New median age: ', median)
print('New mean age: ', init_clean['age_at_incident'].mean())

Number of records with missing/incorrect age:  10991
Original median age:  29.0
Original mean age:  32.10448913979419
New median age:  29.0
New mean age:  32.058614903926305


/Users/AKR/HackLife/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


In [9]:
'''
Convert time features to datetime objects
'''

#list of time features
time_features = ['event_date', 'incident_begin_date', 'arrest_date', 
                 'received_date', 'arraignment_date', 'incident_end_date']

for col in time_features:
    #init_datetime[col] = pd.to_datetime(init_datetime[col], format='%m/%d/%Y %I:%M:%S %p')
    #this version gets the following error: 'OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2912-01-18 00:00:00'

    #turn nan to '01/01/1900 12:00:00 AM' so can process them as strings
    init_clean.loc[init_clean[col].isnull(), col] = '01/01/1900 12:00:00 AM'

    #make sure everything is strings
    init_clean[col] = init_clean[col].astype(str)

    #change type to datetime
    init_clean[col] = init_clean[col].map(lambda x: pd.datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))


In [10]:
'''
Clean gender column
'''

#look at unique values
print(init_clean['gender'].unique())

#how many are messed up?
print('Number of records with gender=Male name, no gender given: ', len(init_clean[init_clean['gender']=='Male name, no gender given']))
print('Number of records with gender=Unknown: ', len(init_clean[init_clean['gender']=='Unknown']))
print('Number of records with gender=Unknown Gender: ', len(init_clean[init_clean['gender']=='Unknown Gender']))
print('Number of records with null Gender: ', len(init_clean[init_clean['gender'].isnull()]))

#turning all except 'Male' and 'Female' to 'Unknown'
init_clean.loc[init_clean['gender'].isnull(), 'gender'] = 'Unknown'
init_clean.loc[init_clean['gender']=='Male name, no gender given', 'gender'] = 'Unknown'
init_clean.loc[init_clean['gender']=='Unknown Gender', 'gender'] = 'Unknown'


['Male' 'Female' nan 'Male name, no gender given' 'Unknown Gender'
 'Unknown']
Number of records with gender=Male name, no gender given:  4
Number of records with gender=Unknown:  7
Number of records with gender=Unknown Gender:  2
Number of records with null Gender:  2722


In [11]:
'''
Clean race column
'''

#look at unique values
print(init_clean['race'].unique())

#change all to lower, to combine ASIAN and Asian
init_clean['race']=init_clean['race'].str.lower()

#look at unique values again
print(init_clean['race'].unique())

#how many are messed up?
print(init_clean['race'].value_counts())
print('Number of records with null race: ', len(init_clean[init_clean['race'].isnull()]))

#change nan to 'unknown'
init_clean['race'].fillna('unknown', inplace = True)

['Black' 'HISPANIC' 'White [Hispanic or Latino]'
 'White/Black [Hispanic or Latino]' 'White' nan 'Unknown' 'Asian'
 'Biracial' 'American Indian' 'Albino' 'ASIAN']
['black' 'hispanic' 'white [hispanic or latino]'
 'white/black [hispanic or latino]' 'white' nan 'unknown' 'asian'
 'biracial' 'american indian' 'albino']
black                               494039
white [hispanic or latino]          128261
white                                97785
hispanic                              9259
asian                                 5017
white/black [hispanic or latino]      4011
unknown                               1119
american indian                        385
biracial                               102
albino                                   1
Name: race, dtype: int64
Number of records with null race:  3825


In [12]:
'''
Convert charge_count to int
'''

#look at unique values of charge_count
print('Unique charge count values: ', init_clean['charge_count'].unique())

#change type to int
init_clean.charge_count = init_clean.charge_count.astype(int)

Unique charge count values:  [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105   1 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 2

In [13]:
'''
Convert all non-id string features to lowercase
'''

string_cols = ['offense_category', 'charge_offense_title', 'chapter', 'act', 
               'section', 'class', 'aoic', 'event', 'gender', 'race', 
               'law_enforcement_agency', 'unit', 'incident_city', 'updated_offense_category']

for col in string_cols:
    init_clean[col]=init_clean[col].str.lower()
    #note that str.lower() automatically ignores null values

In [14]:
'''
Convert all remaining null values to 'unknown'
'''

#where are there still null values?
print(init_clean.columns[init_clean.isna().any()].tolist())
#all null values are in string columns
#replace remaining null values with 'unknown'
#chose 'unknown' because this seems to be what Cook County uses
init_clean.fillna('unknown', inplace = True)

['chapter', 'act', 'section', 'aoic', 'event', 'law_enforcement_agency', 'unit', 'incident_city']


In [15]:
'''
FEATURE ENGINEERING:
Binary 402 Indicator Feature

If an entry in section contains 402, then 402 column value will be 1
Else, it will be 0

'''
# initialize empty column 402 
init_clean["402"] = 0


def sectionconverter(x): 
    if "402" in x: 
        new_row = 1
    else: 
        new_row = 0
    return new_row

init_clean['402'] = init_clean['section'].apply(sectionconverter)

In [16]:
#CHECKPOINT:
#save cleaned dataframe to csv

#init_clean.to_csv('init_clean.csv')


In [17]:
#init_clean = pd.read_csv('init_clean.csv', index_col=0)

# Groupby and Aggregate

In [18]:
'''
Create dummy variables for categorical columns
'''

init_clean_copy = init_clean.copy()

# create list of categorical columns to get dummies for
# including all categorical variables
# excluding ID numbers, numerical features, datetime features, binary features
cat_cols = ['offense_category', 'charge_offense_title', 'chapter', 'act', 
            'section', 'class', 'aoic', 'event', 'gender', 'race', 
            'law_enforcement_agency', 'unit', 'incident_city', 'updated_offense_category']
dummy_init = pd.get_dummies(init_clean_copy, columns=cat_cols)

del init_clean_copy

In [19]:
'''
Drop datetime features and the ID numbers we aren't using
Datetime features will be reintroduced after aggregation

list of time features defined in cleaning section:
time_features = ['event_date', 'incident_begin_date', 'arrest_date', 
                 'received_date', 'arraignment_date', 'incident_end_date']
'''

#list of id features, including primary_charge flag
id_features = ['case_id', 'primary_charge', 'charge_id', 'charge_version_id']

to_drop = time_features + id_features
dummy_init = dummy_init.drop(columns=to_drop)

print(dummy_init.shape)

# trying to save to csv
# dummy_init.to_csv('dummy_init.csv')

(743804, 5423)


In [20]:
'''
Group by CP ID
'''

init_grouped = dummy_init.groupby('case_participant_id')

In [21]:
'''
Preparing for aggregation
Checking which features are the same for every unique CP ID
'''

clean_groups = init_clean.groupby('case_participant_id')
same_within_group = []
diff_within_group = []
for col in init_clean.columns:
    if any(clean_groups[col].nunique()>1):
        diff_within_group.append(col)
    else:
        same_within_group.append(col)

print('same: ', same_within_group)
print('different: ', diff_within_group)

same:  ['case_id', 'case_participant_id', 'offense_category', 'event', 'event_date', 'age_at_incident', 'gender', 'race', 'incident_begin_date', 'incident_end_date', 'arrest_date', 'law_enforcement_agency', 'unit', 'incident_city', 'received_date', 'arraignment_date', 'updated_offense_category', 'age_over_100', 'age_unknown']
different:  ['primary_charge', 'charge_id', 'charge_version_id', 'charge_offense_title', 'chapter', 'act', 'section', 'class', 'aoic', 'charge_count', '402']


In [22]:
'''
Creating dictionary for aggregation

First, create "short" dictionary:
Keys are columns in initiation,
Values are aggregation methods
'''

agg_dict_short = {}

#we take the median for anything that's always the same within unique CP ID groups
#excluding datetime features and case_id
#note that every individual value should be equal to the median
to_median = ['offense_category', 'event', 'age_at_incident', 
             'gender', 'race','law_enforcement_agency', 'unit', 'incident_city',
             'updated_offense_category', 'age_over_100', 'age_unknown']
agg_dict_short = {x : 'median' for x in to_median}

#we take the highest charge_count to represent the total number of charges
agg_dict_short['charge_count'] = 'max'

#we take the highest "402" to maintain binary feature
agg_dict_short['402'] = 'max'

#list of categorical features that vary within unique case_participant_id groups
#excluding id numbers
to_sum = ['charge_offense_title', 'chapter', 'act', 'section', 'class', 'aoic']
#we will sum these features when we aggregrate their dummies
for item in to_sum:
    agg_dict_short[item] = 'sum'

print(agg_dict_short)

{'offense_category': 'median', 'event': 'median', 'age_at_incident': 'median', 'gender': 'median', 'race': 'median', 'law_enforcement_agency': 'median', 'unit': 'median', 'incident_city': 'median', 'updated_offense_category': 'median', 'age_over_100': 'median', 'age_unknown': 'median', 'charge_count': 'max', '402': 'max', 'charge_offense_title': 'sum', 'chapter': 'sum', 'act': 'sum', 'section': 'sum', 'class': 'sum', 'aoic': 'sum'}


In [23]:
'''
Creating dictionary for aggregation

Use "short" dictionary to create "long" dictionary:
Keys are columns in dummy_init,
Values are aggregation methods
'''

agg_dict_long = {}

#loop through every key, val pair in agg_dict_short
for key, val in agg_dict_short.items():

    #check if key is not in the list of cat_cols that got turned into dummy variables
    if key not in cat_cols:

        #add key, val pair to agg_dict_long
        agg_dict_long[key] = val

    else:

        #loop through every column in dummy_init
        for col in dummy_init:

            #check if the key is a substring of the dummy_init column name
            if key+'_' in col:

                #add val to agg_dict_long with the dummy_init column name as new key
                agg_dict_long[col] = val

del dummy_init

In [24]:
'''
Aggregate the post-dummy groupby object
'''

init_squish = init_grouped.aggregate(agg_dict_long)
# note that index is now cp id

In [25]:
'''
Append MHI column to aggregated dataframe
'''

#initialize MHI column to false
init_squish['MHI'] = 0

#change MHI to 1 for the cp ids which appear in MHI_true
init_squish.loc[init_squish.index.isin(MHI_true.case_participant_id.values), 'MHI']=1

print('Number of positive cases in aggregated dataset: ', len(init_squish[init_squish['MHI']==1]))
print('Number of negative cases in aggregated dataset: ', len(init_squish[init_squish['MHI']==0]))

Number of positive cases in aggregated dataset:  2212
Number of negative cases in aggregated dataset:  286882


In [26]:
'''
Investigate discrepancy between number of positive cases 
in MHI_true vs. init_squish
'''

missing_positives = list(set(MHI_true.case_participant_id.values).difference(set(init_squish.index.values)))
print(len(missing_positives))
print(any(initiation.case_participant_id.isin(missing_positives)))
#'False' confirms that none of the 'missing positives' are present in initiation

227
False


In [27]:
# trying to save to csv
# init_squish.to_csv('init_squish.csv')
# too big, not working

In [28]:
'''
Adding datetime features to aggregated dataframe
Could not be aggregated because they are non-numeric
Easily reintroduced because they are always the same within unique CP ID groups

list of time features defined in cleaning section:
time_features = ['event_date', 'incident_begin_date', 'arrest_date', 
                 'received_date', 'arraignment_date', 'incident_end_date']
                 
Also adding location-related categorical features
As with time features, these are always the same within unique CP ID groups
Currently duplicates the information from dummy variables
Will be used for feature engineering, then removed
'''

#making df copies to work on
total_df = init_squish.copy()
time_df = init_clean.copy()

#make list of columns to keep/delete from init_clean
#keep time features, cp id, non-dummy location features
to_keep = time_features + ['case_participant_id', 'incident_city', 'unit']
to_delete  = list(set(time_df.columns) - set(to_keep))

#drop columns in to_delete list
time_df.drop(columns=to_delete, inplace=True)

#drop rows with with duplicate cp id's
time_df.drop_duplicates(subset='case_participant_id', keep='first', inplace=True)

#confirm that time_df now has as many rows as init_squish
print(len(init_squish)==len(time_df))
#test works, prints True

del init_squish
del init_clean

#set cp id to index of time_df
time_df.set_index('case_participant_id', inplace=True)

#join our time and location columns from time_df onto total_df
total_df = total_df.join(time_df, on='case_participant_id')

del time_df

print(total_df.shape)

True
(289094, 5431)


# Feature Engineering

In [29]:
'''
See end of cleaning section for binary '402' indicator feature
'''

"\nSee end of cleaning section for binary '402' indicator feature\n"

In [30]:
''' 
FEATURE ENGINEERING: Weekday

Creating binary weekday column: 0 = Sat/Sun, 1 Otherwise'''

arrest_date_dt = pd.to_datetime(total_df.arrest_date) #converting to datetime
arrest_date_dt = arrest_date_dt.replace(datetime.datetime(1900, 1, 1, 0, 00), np.nan) #replacing 1900 with NaT

total_df['weekday'] = arrest_date_dt #instantiating new column
total_df['weekday'] = np.where(total_df.weekday.isnull(), np.nan, total_df.weekday.dt.dayofweek < 5)
# converting NaT to np.nan, checking if weekday ^

#total_df['weekday'].loc[883102444154] #here is a nan for testing
#how should this handle NaN values of 1900? -akr .. fixed, now 1900 dates are NaN -kcm
# QUESTION: these are floats, do they need to be ints? not sure how to do this with nans

total_df.head()

,offense_category_aggravated assault police officer,offense_category_aggravated assault police officer firearm,offense_category_aggravated battery,offense_category_aggravated battery police officer,offense_category_aggravated battery police officer firearm,offense_category_aggravated battery with a firearm,offense_category_aggravated discharge firearm,offense_category_aggravated dui,offense_category_aggravated fleeing and eluding,offense_category_aggravated identity theft,...,MHI,event_date,incident_begin_date,incident_end_date,arrest_date,unit,incident_city,received_date,arraignment_date,weekday
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,0,0,0,0,0,0,0,0,0,0,...,0,2011-06-20 00:00:00,2007-03-12,1900-01-01,2011-05-22 18:51:00,unknown,unknown,2011-05-24,2011-07-11 00:00:00,0.0
272161011760,0,0,0,0,0,0,0,0,0,0,...,0,1900-01-01 00:00:00,2009-06-25,1900-01-01,2009-07-14 14:34:00,unknown,unknown,2012-01-27,1900-01-01 00:00:00,1.0
864286527653,0,0,0,0,0,0,0,0,0,0,...,0,2010-04-28 00:00:00,2010-04-07,1900-01-01,2010-04-07 20:44:00,unknown,unknown,2011-01-31,2010-05-19 00:00:00,1.0
882206007016,0,0,0,0,0,0,0,0,0,0,...,0,2011-01-05 00:00:00,2010-12-15,1900-01-01,2010-12-15 20:40:00,unknown,chicago,2011-01-31,2011-01-31 00:00:00,1.0
882242005211,0,0,0,0,0,0,0,0,0,0,...,0,2011-08-05 00:00:00,2011-07-17,1900-01-01,2011-07-17 19:05:00,unknown,chicago,2011-07-17,1900-01-01 00:00:00,0.0


In [31]:
'''
FEATURE ENGINEERING: Season 

'''
from datetime import date, datetime

arrest_date_dt = pd.to_datetime(total_df.arrest_date) #converting to datetime
total_df['day_of_year'] = arrest_date_dt.dt.dayofyear # making day of year column- ordinal date

#checking for various seasons
total_df['season'] = np.where(total_df['day_of_year'] <= 80, 'winter', total_df['day_of_year'])
total_df['season'] = np.where((173 > total_df['day_of_year']) &  (total_df['day_of_year'] > 80), 'spring', total_df['season'])
total_df['season'] = np.where((267 > total_df['day_of_year']) &  (total_df['day_of_year'] >= 173), 'summer', total_df['season'])
total_df['season'] = np.where((356 > total_df['day_of_year']) &  (total_df['day_of_year'] >= 267), 'fall', total_df['season'])
total_df['season'] = np.where(356 <= total_df['day_of_year'], 'winter', total_df['season'])

total_df = total_df.drop(columns = 'day_of_year') # dropping day of year column  
total_df = pd.get_dummies(total_df, columns = ['season'])
total_df.head()

,offense_category_aggravated assault police officer,offense_category_aggravated assault police officer firearm,offense_category_aggravated battery,offense_category_aggravated battery police officer,offense_category_aggravated battery police officer firearm,offense_category_aggravated battery with a firearm,offense_category_aggravated discharge firearm,offense_category_aggravated dui,offense_category_aggravated fleeing and eluding,offense_category_aggravated identity theft,...,arrest_date,unit,incident_city,received_date,arraignment_date,weekday,season_fall,season_spring,season_summer,season_winter
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,0,0,0,0,0,0,0,0,0,0,...,2011-05-22 18:51:00,unknown,unknown,2011-05-24,2011-07-11 00:00:00,0.0,0,1,0,0
272161011760,0,0,0,0,0,0,0,0,0,0,...,2009-07-14 14:34:00,unknown,unknown,2012-01-27,1900-01-01 00:00:00,1.0,0,0,1,0
864286527653,0,0,0,0,0,0,0,0,0,0,...,2010-04-07 20:44:00,unknown,unknown,2011-01-31,2010-05-19 00:00:00,1.0,0,1,0,0
882206007016,0,0,0,0,0,0,0,0,0,0,...,2010-12-15 20:40:00,unknown,chicago,2011-01-31,2011-01-31 00:00:00,1.0,1,0,0,0
882242005211,0,0,0,0,0,0,0,0,0,0,...,2011-07-17 19:05:00,unknown,chicago,2011-07-17,1900-01-01 00:00:00,0.0,0,0,1,0


In [32]:
'''
FEATURE ENGINEERING: Incident length

akr notes for kcm: 
idk how to catch the null values in start/end. (so we can fill in 0 for incident_length)
might be easier to change the way we're converting nulls to datetime back in the data prep section
my 1900 thing is pretty stupid, i just didnt know how else to do it
'''

# convert begin and end dates to datetime
begin_date_dt = pd.to_datetime(total_df.incident_begin_date)
begin_date_dt = begin_date_dt.replace(datetime(1900, 1, 1, 0, 00), np.nan) #replacing 1900 with NaT
end_date_dt = pd.to_datetime(total_df.incident_end_date)
end_date_dt = end_date_dt.replace(datetime(1900, 1, 1, 0, 00), np.nan) #replacing 1900 with NaT

# create incident_length column by subtracting, this will be NaT if either is NaT
total_df['incident_length'] = (end_date_dt - begin_date_dt)

# This should replace NaT with 0 
total_df['incident_length'] = np.where(total_df.incident_length.isnull(), 0, total_df.incident_length)

# Convert all timestamps to days 
total_df['incident_length'] = total_df.incident_length.dt.days

# delete everything after the space 
# assumes that the string is formatted '_____ days ...'
# might be a better way to do this part using datetime.days, but i couldnt get it to work
#total_df['incident_length'] = total_df['incident_length'].map(lambda x: x.partition(' ')[0])
#total_df['incident_length'] = total_df['incident_length'].astype(int)

#Testing
# Has end date but not begin date: 184594, 58125, 223173
# end_date_dt.iloc[58125] - begin_date_dt.iloc[58125] #testing- has end_date but not begin
# Has begin date but not end: 1256, 920, many more.. 

In [33]:
print(type(total_df.incident_length.iloc[245]))
total_df.incident_length.value_counts()

<class 'numpy.int64'>


0       279416
1         1998
2          394
3          245
4          197
         ...  
2131         1
338          1
3411         1
1618         1
8188         1
Name: incident_length, Length: 1123, dtype: int64

In [34]:

#feature engineering:
#geoencode incident_city

# columns related to location: incident_city, unit
uniqueCities = total_df['incident_city'].unique() # 264 ? cities

# add state to city names in unique list
uniquePlaces =  [(i + ", Illinois") for i in uniqueCities]
# drop Unknown -- don't "locate" cities indicated as unknown
uniquePlaces = uniquePlaces[1:]
# convert to df
uniquePlaces_df = pd.DataFrame(uniquePlaces)

# geoencode unique list
# 1 - conveneint function to delay between geocoding calls
locator = Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2 - create location column
uniquePlaces_df['location'] = uniquePlaces_df[0].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
uniquePlaces_df['point'] = uniquePlaces_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split tuple into longitude and latitude (and altitude)
uniquePlaces_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(uniquePlaces_df['point'].tolist(), index=uniquePlaces_df.index)

# remove state from city names
uniquePlaces_df[0]=uniquePlaces_df[0].str[:-10]


# set default values (will remain 0 if incidenct_city is unknown)
total_df['latitude'] = 0
total_df['longitude'] = 0

# assign appropriate long/lat
for i in range(len(uniquePlaces_df)):

  samePlace = uniquePlaces_df[0][i]
  lat = uniquePlaces_df['latitude'][i]
  lon = uniquePlaces_df['longitude'][i]

  # update lat and lon if df['incident_city']==uniquePlaces_df['0']
  total_df['latitude'] = np.where((total_df.incident_city == samePlace), lat, total_df.latitude)
  total_df['longitude'] = np.where((total_df.incident_city == samePlace), lon, total_df.longitude)

del uniquePlaces_df

RateLimiter caught an error, retrying (0/2 tries). Called with (*('maywood, Illinois',), **{}).
Traceback (most recent call last):
  File "/Users/AKR/HackLife/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/AKR/HackLife/anaconda3/lib/python3.7/http/client.py", line 1244, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/AKR/HackLife/anaconda3/lib/python3.7/http/client.py", line 1290, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/AKR/HackLife/anaconda3/lib/python3.7/http/client.py", line 1239, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/AKR/HackLife/anaconda3/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/AKR/HackLife/anaconda3/lib/python3.7/http/client.py", line 966, in send
    self.connect()
  File "/Users/AKR/HackLi

In [35]:
# print df -- check for chicago coordinates
total_df.head(20)

,offense_category_aggravated assault police officer,offense_category_aggravated assault police officer firearm,offense_category_aggravated battery,offense_category_aggravated battery police officer,offense_category_aggravated battery police officer firearm,offense_category_aggravated battery with a firearm,offense_category_aggravated discharge firearm,offense_category_aggravated dui,offense_category_aggravated fleeing and eluding,offense_category_aggravated identity theft,...,received_date,arraignment_date,weekday,season_fall,season_spring,season_summer,season_winter,incident_length,latitude,longitude
case_participant_id,,,,,,,,,,,,,,,,,,,,,
260122253823,0,0,0,0,0,0,0,0,0,0,...,2011-05-24,2011-07-11 00:00:00,0.0,0,1,0,0,0,0.000000,0.000000
272161011760,0,0,0,0,0,0,0,0,0,0,...,2012-01-27,1900-01-01 00:00:00,1.0,0,0,1,0,0,0.000000,0.000000
864286527653,0,0,0,0,0,0,0,0,0,0,...,2011-01-31,2010-05-19 00:00:00,1.0,0,1,0,0,0,0.000000,0.000000
882206007016,0,0,0,0,0,0,0,0,0,0,...,2011-01-31,2011-01-31 00:00:00,1.0,1,0,0,0,0,41.875562,-87.624421
882242005211,0,0,0,0,0,0,0,0,0,0,...,2011-07-17,1900-01-01 00:00:00,0.0,0,0,1,0,0,41.875562,-87.624421
882325141108,0,0,0,0,0,0,0,0,0,0,...,2011-01-31,2011-01-31 00:00:00,1.0,1,0,0,0,0,41.875562,-87.624421
882415546999,0,0,0,0,0,0,0,0,0,0,...,2011-03-18,2011-05-03 00:00:00,1.0,0,0,0,1,0,41.875562,-87.624421
883042173560,0,0,0,0,0,0,0,0,0,0,...,2011-04-15,2011-01-28 00:00:00,1.0,0,0,0,1,0,41.875562,-87.624421
883053547582,0,0,0,0,0,0,0,0,0,0,...,2011-01-01,2011-02-02 00:00:00,1.0,0,0,0,1,0,41.875562,-87.624421


# Final Data Prep: Scaling, Train/Test Split, Downsampling

In [36]:
''' 
Create final_df from copy of total_df
Removing "unsquishable" categorical columns from total_df that we added back in for engineering
'''

final_df = total_df.copy()

del total_df

to_remove = ['event_date', 'incident_begin_date', 'arrest_date', 'arraignment_date', 'incident_city', 'unit']
final_df = final_df.drop(columns=to_remove)

In [37]:
'''
SCALING / NORMALIZATION

kcm code expanded by akr

turns total_df into final_df, final_df becomes our working copy

akr notes/questions:
we should add all other numeric columns that we create
is it ok that we're normalizing before downsampling?
did i expand this code right? the "since only one column" thing makes me think maybe not
kcm: i think so, thats just what you do if its just one series'''

from sklearn.preprocessing import StandardScaler

age = final_df['age_at_incident'].values.reshape(-1, 1) #reshaping since only one column
scaler = StandardScaler().fit(age)
final_df['age_at_incident'] = scaler.transform(age) 
#print(final_df['age_at_incident'].mean()) #~ 0

charge_count = final_df['charge_count'].values.reshape(-1, 1) #reshaping since only one column
scaler = StandardScaler().fit(charge_count)
final_df['charge_count'] = scaler.transform(charge_count) 

incident_length = final_df['incident_length'].values.reshape(-1, 1) #reshaping since only one column
scaler = StandardScaler().fit(incident_length)
final_df['incident_length'] = scaler.transform(incident_length)
#print(final_df['incident_length'].mean()) #~ 0


# need to add all dummy features created by the to_sum list. will be sparse
# ie, any key for which the value is 'sum' in agg_dict_long
to_sum = ['chapter', 'act', 'section', 'class', 'aoic', 'charge_offense_title']

for variable in to_sum:

  # list of columns starting with variable
  filter_col = [col for col in final_df if col.startswith(variable)]

  for column in filter_col:
    var = final_df[column].values.reshape(-1, 1) 
    scaler = StandardScaler().fit(var)
    final_df[column] = scaler.transform(var) 


KeyboardInterrupt: 

In [ ]:
# Save final_df to drive

# from google.colab import drive
# drive.mount('drive')

# final_df.to_csv('final_df.csv')
# !cp final_df.csv drive/My\ Drive/

In [ ]:
'''
Split Data into Train / Validate / Test Sets

current method: 'timeline'
ensures that we only ever predict forward in time

should be executed on scaled df, ie final_df
'''


def get_train_val_test (df, method, train_size, test_size, val_size): 
  #takes in df (assumes 'MHI' is target variable)
  #methods are strings, defined below
  #takes in train, test, and val sizes (fractions of 1)
  #returns xtrain, xval, xtest, ytrain, yval, ytest

  #current method: 'timeline'

  #list of methods that are finished and ready to be used
  possible_methods = ['simple', 'timeline']

  #check that inputted method is in possible_methods
  if method not in possible_methods:
      print('error. possible methods: ', possible_methods)
      return 
  
  if method=='simple':
    #static validation set, random sampling via shuffle
    #uses train_size, test_size, val_size

    #check that specificed sizes sum to 1
    if train_size + test_size+val_size != 1:
      print('error: train_size + test_size + val_size must sum to 1')
      return 

    df.drop('received_date', axis=1)
      
    #set x and y
    x = df.drop('MHI', axis=1)
    y = df['MHI']

    #first split test from the other 2
    xtrainval, xtest, ytrainval, ytest = train_test_split(x, y, test_size=test_size, shuffle=True, random_state=2019)
    
    #second split train and val
    xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=val_size/(val_size+train_size), shuffle=True, random_state=2019)
    return xtrain, xval, xtest, ytrain, yval, ytest


  if method=='timeline':
    #static validation set, divisions based on timeline
    #should reduce leakage
    #danger of bias bc training set will have higher proportion of "finalized" rows

    #check that specificed sizes sum to 1
    if train_size + test_size + val_size != 1:
      print('error: train_size + test_size + val_size must sum to 1')
      return 

    #sort ascending by received date
    sorted_df = df.sort_values(by=['received_date'])
    sorted_df = sorted_df.drop('received_date')

    #set x and y
    x = sorted_df.drop('MHI', axis=1)
    y = sorted_df['MHI']

    total_size = len(x)

    #first train_size % indices in train
    xtrain = x.iloc[:int(round(train_size*total_size))]
    ytrain = y.iloc[:int(round(train_size*total_size))]

    #next val_size % indices in val
    xval = x.iloc[int(round(train_size*total_size)):int(round((train_size+val_size)*total_size))]
    yval = y.iloc[int(round(train_size*total_size)):int(round((train_size+val_size)*total_size))]

    #final test_size % indices in test
    xtest = x.iloc[int(round((train_size+val_size)*total_size)):-1]
    ytest = y.iloc[int(round((train_size+val_size)*total_size)):-1]

    return xtrain, xval, xtest, ytrain, yval, ytest

  if method=='walk':
    #walk forward
    return

In [ ]:
# execute the split on final_df
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test (final_df, 'timeline', 0.7, 0.15, 0.15)

In [ ]:
'''
Downsampling the training set

at function tweaked by akr
'''

from random import sample 

def downsample(df, pct_MHI1): 
  # takes in percentage from 1 to 50
  # samples all MHI==1 cases
  # samples from MHI==0 cases such that downsampled_df has pct_MHI1 % positive cases

  # split into MHI 1 and MHI 0 
  MHI1 = df[df['MHI'] == 1]
  MHI0 = df[df['MHI'] == 0]

  # count number of MHIs
  count_MHI1 = len(MHI1)

  # compute number of negative cases to sample
  num_MHI0 = count_MHI1 * int(round((100-pct_MHI1)/pct_MHI1))

  # sample from negative cases
  MHI0_sample = MHI0.sample(n=num_MHI0, random_state=42)

  # append sampled negative cases to all positive cases
  downsampled_df = MHI1.append(MHI0_sample)

  return downsampled_df

training_df = X_train.join(y_train)

# passing 50 to downsample function means training set will have 50% positive cases
# we can vary this downsampling call between 1% and 50%
downsampled_training_df = downsample(training_df, 50)

X_train_down = downsampled_training_df.drop('MHI', axis=1)
y_train_down = downsampled_training_df['MHI']

# Random Forest Baseline Model

> Indented block



In [ ]:
'''
KCM Random Forest Baseline
'''

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score

non_baseline = ['402', 'weekday', 'season', 'incident_length', 'age_over_100', 'age_unknown']
# 'latitude', 'longitude'

base_X_train_down = X_train_down.drop(columns=non_baseline)
base_X_val = X_val.drop(columns=non_baseline)
base_X_test.drop(columns=non_baseline)

clf = RandomForestClassifier(random_state=0).fit(base_X_train_down,y_train)
y_pred_proba = clf.predict_proba(base_X_val)
y_pred = clf.predict(base_X_val)
print("Accuracy:", metrics.accuracy_score(y_val, y_pred))
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:,1])
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)

In [ ]:
'''
Feature Importance Plot for Random Forest
'''

import matplotlib.pyplot as plt
ftimp = [x for x in zip(X.columns, clf.feature_importances_)] #list of tuples
ftimp.sort(key = lambda x: x[1]) #sorting based on feature importance
ftimp = [x for x in ftimp if x[1] > 0.01] #theres a ton above 0
plt.figure(figsize=(15,15))
plt.bar(*zip(*ftimp)) #unzipping ftimp
plt.xticks(rotation='vertical')
plt.show()

# Data Analysis

In [ ]:
''' 
FEATURE ENGINEERING FOR DATA ANAYLSIS:
create days since received column

subtracting from data upload date of July 26,2019

chose received because it's the only "start" date without missing values
'''

upload_date = pd.to_datetime(pd.datetime.strptime('7/26/2019', '%m/%d/%Y'))
received_date_dt = pd.to_datetime(total_df.received_date)
total_df['days_since_received'] = (upload_date - received_date_dt).astype(str)
total_df['days_since_received'] = total_df['days_since_received'].map(lambda x: x.partition(' ')[0])
total_df['days_since_received'] = total_df['days_since_received'].astype(int)

In [ ]:
'''
Data Analysis:
Distribution of MHI over Time
'''

import matplotlib.pyplot as plt
%matplotlib inline


def get_portion_true (decile):
  #returns the portion of MHI==1 out of the total decile
  num_true = len(decile[decile['MHI']==1])
  return (num_true/len(decile))

def plot_deciles(df, col):

  #get the total number of rows in the df
  n = len(df)

  #sort df by column of interest
  sorted_df = df.sort_values(by=[col])

  #create deciles, ie bins of the column of interest
  decile_1 = sorted_df.iloc[0:round(n/10)]
  decile_2 = sorted_df.iloc[round(n/10):round(2*n/10)]
  decile_3 = sorted_df.iloc[round(2*n/10):round(3*n/10)]
  decile_4 = sorted_df.iloc[round(3*n/10):round(4*n/10)]
  decile_5 = sorted_df.iloc[round(4*n/10):round(5*n/10)]
  decile_6 = sorted_df.iloc[round(5*n/10):round(6*n/10)]
  decile_7 = sorted_df.iloc[round(6*n/10):round(7*n/10)]
  decile_8 = sorted_df.iloc[round(7*n/10):round(8*n/10)]
  decile_9 = sorted_df.iloc[round(8*n/10):round(9*n/10)]
  decile_10 = sorted_df.iloc[round(9*n/10):n]

  #list of deciles
  deciles = [decile_1, decile_2, decile_3, decile_4, decile_5, 
           decile_6, decile_7, decile_8, decile_9, decile_10]

  #create list of portions
  portions = [get_portion_true(x) for x in deciles]

  #create string labels of earliest arrest date in decile
  labels = [str(x.iloc[0][col]) for x in deciles]

  #plot
  fig, ax = plt.subplots()
  ax.bar(labels, portions, width=.55)
  ax.set_xticklabels(labels, rotation = 90)
  ax.set_ylabel('Portion of records with MHI==1')
  plt.title('MHI by {}'.format(col))
  plt.show()

  return

time_features_to_plot = ['event_date', 'received_date', 'arrest_date', 'incident_begin_date']

for column in time_features_to_plot:
  plot_deciles(total_df, column)


In [ ]:
'''
CENSORING PROBLEM ANALYSIS
Looking at the degree to which the "rolling in" effects our base rates
Should ultimately select cut-off heuristic (per Brian)

Need to create scatter plot
X axis: age of case
Y axis: MHI base rate
how to do this, though? binning?

Note: Need to do research on censoring problem
Brian told me to research censoring in survival analysis
'''


In [ ]:
''' 
Data Analysis:
Looking at MHI Base Rates
Across Train/Validation/Test
'''
  
training_df = X_train.join(y_train)
val_df = X_val.join(y_val)
train_and_val_df = pd.concat([training_df, val_df], verify_integrity=True)
test_df = X_val.join(y_val)


print ("Training base rate: ", get_portion_true(training_df))
print ("Validation base rate: ", get_portion_true(val_df))
print ("Training+Validation base rate: ", get_portion_true(train_and_val_df))
print ("Test base rate: ", get_portion_true(test_df))